# Open search data uploader.
### Important notice: Just run this notebook one time. Once you have the data on the server, it is not necessary to run this again.

In [1]:
%cd ..
from opensearchpy import OpenSearch, helpers
import pandas as pd
from pprint import pp
from dotenv import load_dotenv
from tqdm import tqdm
import os
def ppr(resp):
    pp(resp.raw)

c:\ulb\SearchEngines_benchmark\venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


c:\ulb\SearchEngines_benchmark


#### Connecting to the server

In [2]:
load_dotenv()

# Recupera la variabile della password dall'ambiente
OPENSEARCH_INITIAL_ADMIN_PASSWORD = os.getenv("OPENSEARCH_INITIAL_ADMIN_PASSWORD")
OPENSEARCH_URL = os.getenv("OPENSEARCH_URL")  
# Verifica che la password sia stata caricata correttamente
if OPENSEARCH_INITIAL_ADMIN_PASSWORD:
    print("Password correct.")
else:
    print("Error in the password")

Password correct.


#### Remember to run the docker container. Sometimes after start running the container, you should wait 1 minute to actually connect to the server.

In [3]:
client = OpenSearch(
    hosts=[OPENSEARCH_URL],
    http_auth=("admin", OPENSEARCH_INITIAL_ADMIN_PASSWORD),
    use_ssl=False,
    verify_certs=False,
    ssl_assert_hostname=False,
    ssl_show_warn=False
)
try:
    info = client.info()
    print("Connection to OpenSearch succesful!")
except Exception as e:
    print(f"Error in the connection: {e}")

Connection to OpenSearch succesful!


## Uploading post data

In [4]:
#df = pd.read_csv('data/Users.csv')

In [5]:
#df.columns

Index(['Unnamed: 0', 'Id', 'AboutMe', 'Age', 'CreationDate', 'DisplayName',
       'DownVotes', 'EmailHash', 'LastAccessDate', 'Location', 'Reputation',
       'UpVotes', 'Views', 'WebsiteUrl', 'AccountId'],
      dtype='object')

In [9]:
# df[~df['Location'].isna()]

In [8]:
def generator_post(json_chunk, index_):
    for line in json_chunk:
        yield {
            "_index": index_,
            "_id": line.get('Id'),
            "_source": {
                "Body": line.get("Body",""),
                "CommentCount": line.get("CommentCount",""),
                "CreationDate": line.get("CreationDate",""),
                "OwnerUserId": line.get("OwnerUserId",""),
                "ClosedDate": line.get("ClosedDate",""),
                "Title": line.get("Title",""),
                "Tags": line.get("Tags",""),
                "FavoriteCount": line.get("FavoriteCount",""),
                "Score": line.get("Score","")
            }
        }
        
def generator_comments(json_chunk, index_):
    for line in json_chunk:
        yield {
            "_index": index_,
            "_id": line.get('Id'),
            "_source": {
                "Text": line.get("Text",""),
                "Score": line.get("Score",""),
                "CreationDate": line.get("CreationDate",""),
                "UserId": line.get("UserId","")
            }
        }


def generator_user(json_chunk, index_):
    for line in json_chunk:
        yield {
            "_index": index_,
            "_id": line.get('Id'),
            "_source": {
                "AboutMe": line.get("AboutMe",""),
                "CreationDate": line.get("CreationDate",""),
                "DisplayName": line.get("DisplayName",""),
                "DownVotes": line.get("DownVotes",""),
                "LastAccessDate": line.get("LastAccessDate",""),
                "UpVotes": line.get("UpVotes",""),
                "Reputation": line.get("Reputation",""),
                "Location": line.get("Location","")
            }
        }

def from_chunk_to_client_post(chunk,index_):
    json_chunk = chunk.to_dict("records")
    return generator_post(json_chunk,index_)

def from_chunk_to_client_user(chunk,index_):
    json_chunk = chunk.to_dict("records")
    return generator_user(json_chunk,index_)

def from_chunk_to_client_comments(chunk,index_):
    json_chunk = chunk.to_dict("records")
    return generator_comments(json_chunk,index_)

In [10]:
# #index name to delete
# index_name = "users"  # Replace with the name of the index you want to delete

# # Delete the index
# try:
#     response = client.indices.delete(index=index_name)
#     print(f"Index '{index_name}' deleted successfully!")
# except Exception as e:
#     print(f"Error deleting index '{index_name}': {e}")

Index 'users' deleted successfully!


### Post index

In [54]:
#df_reduced = df[['Id', 'Body', 'CommentCount', 'CreationDate', 'OwnerUserId', 'ClosedDate', 'Title', 'Tags','FavoriteCount', 'Score' ]]

In [55]:
#df_reduced

In [56]:
#df_reduced[df_reduced['Score'].isna()]

In [57]:
client.indices.create(
    index="posts",
    body={
        "settings": {
            "analysis": {
                "analyzer": {
                    "custom_text_analyzer": {
                        "type": "standard",
                        "stopwords": "_english_"
                    }
                }
            }
        },
        "mappings": {
            "properties": {
                "Body": {
                    "type": "text",
                    "analyzer": "custom_text_analyzer",
                    "fielddata": True
                },

                "CommentCount": {"type": "long"},
                "CreationDate": {
                    "type": "date",
                    "format": "yyyy-MM-dd HH:mm:ss.SSS"  
                },
                "OwnerUserId": {"type": "long"},
                "ClosedDate": {
                    "type": "date",
                    "format": "yyyy-MM-dd HH:mm:ss.SSS"  
                },
                "Title": {
                    "type": "text",
                     "analyzer": "custom_text_analyzer",
                    "fielddata": True
                },
                "Tags": {
                        "type": "text",
                        "analyzer": "custom_text_analyzer",
                        "fielddata": True
                },
                "FavoriteCount": {"type": "long"},
                "Score": {"type": "long"}
            }
        }
    }
)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'posts'}

In [58]:
columns = ['Id', 'Body', 'CommentCount', 'CreationDate', 'OwnerUserId', 'ClosedDate', 'Title', 'Tags','FavoriteCount', 'Score']
for chunk in tqdm(pd.read_csv('data/Posts.csv', chunksize=1000)):
    chunk['ClosedDate'] = chunk['ClosedDate'].fillna("2070-01-01 00:00:00.000")
    chunk['Title'] = chunk['Title'].fillna("Nan")
    chunk['Tags'] = chunk['Tags'].fillna("Nan")
    gen = from_chunk_to_client_post(chunk[columns], index_='posts')
    res = helpers.bulk(client, gen)

0it [00:00, ?it/s]

1566it [05:06,  5.10it/s]


In [60]:
mapping = client.indices.get_mapping(index="posts")
mapping

{'posts': {'mappings': {'properties': {'Body': {'type': 'text',
     'analyzer': 'custom_text_analyzer',
     'fielddata': True},
    'ClosedDate': {'type': 'date', 'format': 'yyyy-MM-dd HH:mm:ss.SSS'},
    'CommentCount': {'type': 'long'},
    'CreationDate': {'type': 'date', 'format': 'yyyy-MM-dd HH:mm:ss.SSS'},
    'FavoriteCount': {'type': 'long'},
    'OwnerUserId': {'type': 'long'},
    'Score': {'type': 'long'},
    'Tags': {'type': 'text',
     'analyzer': 'custom_text_analyzer',
     'fielddata': True},
    'Title': {'type': 'text',
     'analyzer': 'custom_text_analyzer',
     'fielddata': True}}}}}

In [19]:
# client.indices.create(
#     index='html_posts',
#     body = 
#     {
#     'settings':{
#         "analysis": {
#             "analyzer": {
#                 "html_analyzer": {
#                     "type": "custom",
#                     "tokenizer": "standard",
#                     "char_filter": [
#                         "html_strip"
#                     ]
#                 }
#             }
#         }
#     }, 
#     'mappings':{
#             "properties": {
#                 "Body": {
#                     "type": "text",
#                     "analyzer": "html_analyzer"
#                 },
#                 "CommentCount": {
#                     "type": "integer"
#                 },
#                 "CreationDate": {
#                     "type": "date",
#                     "format": "yyyy-MM-dd HH:mm:ss.SSS"
#                 },
#                 "OwnerUserId": {
#                     "type": "integer"
#                 }
#             }
#         }
#     }
# )


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'html_posts'}

In [ ]:
# columns = ['Id', 'OwnerUserId','Body', 'CommentCount', "CreationDate"]
# for chunk in tqdm(pd.read_csv('data/Posts.csv', chunksize=1000)):
#     gen = from_chunk_to_client_post(chunk[columns], index_='html_posts')
#     res = helpers.bulk(client, gen)

0it [00:00, ?it/s]

1566it [12:16,  2.13it/s]


##### Reindex post to add custom text analyzer

In [ ]:
# client.indices.create(
#     index="posts_analyzed_v2",
#     body={
#         "settings": {
#             "analysis": {
#                 "analyzer": {
#                     "custom_text_analyzer": {
#                         "type": "standard",
#                         "stopwords": "_english_"
#                     }
#                 }
#             }
#         },
#         "mappings": {
#             "properties": {
#                 "Body": {
#                     "type": "text",
#                     "analyzer": "custom_text_analyzer",
#                     "fielddata": True
#                 },
                # "CommentCount": {"type": "long"},
                # "CreationDate": {
                #     "type": "date",
                #     "format": "yyyy-MM-dd HH:mm:ss.SSS"  # Example formats
                # },
                # "OwnerUserId": {"type": "long"}
#             }
#         }
#     }
# )

In [ ]:
# client.reindex(
#     body={
#         "source": {"index": "html_posts"},
#         "dest": {"index": "posts_analyzed_v2"}
#     },
#     request_timeout=1000  # Increase timeout if needed
# )

In [ ]:
# # Fetch and print all indices
# indices = client.cat.indices(format="json")  # Fetch in JSON format for easier processing
# for index in indices:
#     print(f"Index: {index['index']}, Health: {index['health']}, Docs Count: {index['docs.count']}, Size: {index['store.size']}")

## uploading comments

In [ ]:
# #index name to delete
# index_name = "posts_analyzed_v2"  # Replace with the name of the index you want to delete

# # Delete the index
# try:
#     response = client.indices.delete(index=index_name)
#     print(f"Index '{index_name}' deleted successfully!")
# except Exception as e:
#     print(f"Error deleting index '{index_name}': {e}")

Index 'posts_analyzed_v2' deleted successfully!


In [43]:
client.indices.create(
    index="comments",
    body={
        "settings": {
            "analysis": {
                "analyzer": {
                    "custom_text_analyzer": {
                        "type": "standard",
                        "stopwords": "_english_"
                    }
                }
            }
        },
        "mappings": {
            "properties": {
                "Text": {
                    "type": "text",
                    "analyzer": "custom_text_analyzer",
                    "fielddata": True
                },
                "Score": {"type": "long"},
                "CreationDate": {
                    "type": "date",
                    "format": "yyyy-MM-dd HH:mm:ss.SSS"  # Example formats
                },
                "UserId": {"type": "long"},
                "Id": {"type": "long"}
            }
        }
    }
)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'comments'}

In [44]:
mapping = client.indices.get_mapping(index="comments")
print(mapping)

{'comments': {'mappings': {'properties': {'CreationDate': {'type': 'date', 'format': 'yyyy-MM-dd HH:mm:ss.SSS'}, 'Id': {'type': 'long'}, 'Score': {'type': 'long'}, 'Text': {'type': 'text', 'analyzer': 'custom_text_analyzer', 'fielddata': True}, 'UserId': {'type': 'long'}}}}}


In [45]:
columns = ['Id', 'UserId','Text', 'Score', "CreationDate"]
for chunk in tqdm(pd.read_csv('data/Comments.csv', chunksize=1000)):
    # Replace NaN values in the 'UserId' column with a default value, e.g., -1
    chunk['UserId'] = chunk['UserId'].fillna(-1).astype(int)  # Replace with a default value like -1
    # Ensure all columns conform to expected data types
    chunk['Score'] = chunk['Score'].fillna(0).astype(int)
    gen = from_chunk_to_client_comments(chunk[columns], index_='comments')
    res = helpers.bulk(client, gen)

1374it [02:19,  9.86it/s]


# Uploading User

In [11]:
client.indices.create(
    index="users",
    body={
        "settings": {
            "analysis": {
                "analyzer": {
                    "custom_text_analyzer": {
                        "type": "standard",
                        "stopwords": "_english_"
                    }
                }
            }
        },
        "mappings": {
            "properties": {
                "AboutMe": {
                    "type": "text",
                    "analyzer": "custom_text_analyzer",
                    "fielddata": True
                },
                "Score": {"type": "long"},
                "CreationDate": {
                    "type": "date",
                    "format": "yyyy-MM-dd HH:mm:ss.SSS"  # Example formats
                },
                "LastAccessDate": {
                    "type": "date",
                    "format": "yyyy-MM-dd HH:mm:ss.SSS"  # Example formats
                },
                "UserId": {"type": "long"},
                "Id": {"type": "long"},
                "UpVotes": {"type": "long"},
                "DownVotes": {"type": "long"},
                "DisplayName":{"type": "text"},
                "Location":{"type": "text"},
                "Reputation":{"type": "long"}
            }
        }
    }
)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'users'}

In [12]:
columns = ['Id', 'AboutMe', 'CreationDate','DisplayName', 'DownVotes','LastAccessDate','UpVotes','Reputation', 'Location']
for chunk in tqdm(pd.read_csv('data/Users.csv', chunksize=1000)):
    chunk['AboutMe'] = chunk['AboutMe'].fillna("Nan")
    chunk['DisplayName'] = chunk['DisplayName'].fillna("Nan")
    chunk['Location'] = chunk['Location'].fillna("Nan")
    gen = from_chunk_to_client_user(chunk[columns], index_='users')
    res = helpers.bulk(client, gen)

100it [00:12,  8.19it/s]
